# 1. Download repo and datasets
 - This can be run inside a Google Colab 
 - Here you download the repository and the PASCAL-VOC 2007 data

In [0]:
%%time

# Step1 - Get repo
! rm -rf CS4180-DL
! git clone https://github.com/prerakmody/CS4180-DL

# Step2 - Get Dataset
! wget -P /content/CS4180-DL/data/dataset https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
! tar xf /content/CS4180-DL/data/dataset/VOCtrainval_11-May-2012.tar --directory /content/CS4180-DL/data/dataset
! wget -P /content/CS4180-DL/data/dataset https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
! tar xf /content/CS4180-DL/data/dataset/VOCtrainval_06-Nov-2007.tar --directory /content/CS4180-DL/data/dataset
! wget -P /content/CS4180-DL/data/dataset https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
! tar xf /content/CS4180-DL/data/dataset/VOCtest_06-Nov-2007.tar --directory /content/CS4180-DL/data/dataset
! mkdir /content/CS4180-DL/data/weights

! wget -P /content/CS4180-DL/data/weights https://pjreddie.com/media/files/yolov2-voc.weights

 - Here you download some weights

In [0]:
import os
import sys

%load_ext autoreload
%autoreload 2

## Step3 - Add path to main dir
DIR_MAIN = os.path.abspath('CS4180-DL/') #'../../../CS4180-DL/'
sys.path.append(DIR_MAIN)

## Step4 - Download pruned weights from GDrive
from src.utils import download_gdrive

if (0):
    # FILE_ID =  '1dHnUQ8G3GObZSMh9eQ0zUR5wor0ttW3U' # weight-pruning
    FILE_ID = '1I9FE3X5luVTQ0p9j8g1RWy63t5NiRWw_' # filter-pruning
    DIR_WEIGHTS = os.path.join(DIR_MAIN, 'data/weights/pruned')
    DEST_NAME_ZIP   = os.path.join(DIR_WEIGHTS, 'weights-prune-me.zip')
else:
    FILE_ID = '12-5T-ek0CBSIp9SDI03OPKH_kILkvM7E' #retrain-pretrain
    DIR_WEIGHTS = os.path.join(DIR_MAIN, 'data/weights/retrain-pretrain')
    DEST_NAME_ZIP   = os.path.join(DIR_WEIGHTS, 'retrain-pretrain.zip')
    
! mkdir {DIR_WEIGHTS}
DEST_NAME_FILES = DIR_WEIGHTS
download_gdrive(FILE_ID, DEST_NAME_ZIP)
! unzip {DEST_NAME_ZIP} -d {DEST_NAME_FILES}

## Step5 - Generate .txt files for training/validation 
from src.dataloader import setup_VOC
DIR_DATA = os.path.join(DIR_MAIN, 'data/dataset/')
setup_VOC(DIR_DATA)
DIR_DATA_VOC = os.path.join(DIR_DATA, 'VOCdevkit')
! cat {DIR_DATA_VOC}/2007_train.txt {DIR_DATA_VOC}/2007_val.txt {DIR_DATA_VOC}/2012_*.txt > {DIR_DATA_VOC}/voc_train.txt

## Step6 - Check which GPU
! nvidia-smi

# 2. Check GPU
 - check if a GPU is available while using PyTorch

In [0]:
import os
import sys

import torch

dir_main = os.path.abspath('CS4180-DL')
sys.path.append(dir_main)
print (' - In Path : ', sys.path[-1])

USE_GPU = torch.cuda.is_available()
print (' - USE_GPU : ', USE_GPU)

!nvidia-smi

# 3. Predict
 - Here you could predict the the mAP of any pruned weights file for YOLOv2

In [0]:
torch.cuda.empty_cache()
from src.predict import *
from src.nets import *

from src.pruning.weightPruning.methods import weight_prune,quick_filter_prune
from src.pruning.weightPruning.utils import prune_rate

from tensorboardcolab import TensorBoardColab

if (1):
    DIR_PROJ         = 'CS4180-DL'

if (1):
    MODEL            = ''
    MODEL_CFGFILE    = os.path.join(DIR_PROJ, 'data/cfg/github_pjreddie/yolov2-voc.cfg')
    
    #MODEL_WEIGHTFILE = os.path.join(DIR_PROJ, 'data/weights/yolov2-voc.weights') #['0.5' : 0.6366, '0.25' : 0.7, '0.1' : 0.7363]
    MODEL_WEIGHTFILE = os.path.join(DIR_PROJ, 'data/weights/retrain-pretrain/filter-pruned-0.0-retrained_000015.weights')
    #EVAL_PREFIX      = 'iter1_pretrained_'
    EVAL_PREFIX      = 'retrain_pretrain_'
    
    MODEL_LOSS       = RegionLoss()
    
    PASCAL_DIR       = os.path.join(DIR_PROJ, 'data/dataset/VOCdevkit/')
    EVAL_IMAGELIST   = os.path.join(DIR_PROJ, 'data/dataset/VOCdevkit/2007_test.txt')
    EVAL_OUTPUTDIR   = os.path.join(DIR_PROJ, 'eval_data')
    EVAL_OUTPUTDIR_PKL = os.path.join(DIR_PROJ, 'eval_results')

    
    print (' - 0. MODEL       : ', MODEL)
    print (' - 0. MODEL_WEIGHT: ', MODEL_WEIGHTFILE)
    print (' - 0. EVAL_PREFIX : ', EVAL_PREFIX)
    
if (1):
    BATCH_SIZE = 32
    CONF_THRESH = 0.005 # [0.5, 0.25, 0.1, 0.005]
    print (' - 0. BATCH_SIZE  : ', BATCH_SIZE)
    print (' - 0. CONF_THRESH : ', CONF_THRESH)
    
if (1):
    try:
        print (' - 0. Logger      : ', LOGGER)
    except:
        LOGGER = TensorBoardColab()
        print (' - 0. Logger      : ', LOGGER)    
    
    print ('')


valObj = PASCALVOCEval(MODEL, MODEL_CFGFILE, MODEL_WEIGHTFILE, MODEL_LOSS 
                       ,PASCAL_DIR, EVAL_IMAGELIST, EVAL_OUTPUTDIR, EVAL_PREFIX, EVAL_OUTPUTDIR_PKL
                       , LOGGER)
valObj.predict(BATCH_SIZE=BATCH_SIZE, CONF_THRESH=CONF_THRESH)
# valObj._do_python_eval()

# 4. Re-Training
 - Here you set neural model hyperparams, type of pruning and retrain the network to adjust the pruned weights 

In [0]:
%reset
import os
import sys

import torch
from tensorboardcolab import TensorBoardColab

torch.cuda.empty_cache()

if (torch.cuda.is_available()):
    
    if (1):
        DIR_MAIN         = os.path.abspath('CS4180-DL')
        sys.path.append(DIR_MAIN)
        print (' - 0. DIR_MAIN     :  ', DIR_MAIN)
        from src.train import YOLOv2Train
        
    if (1):
        PASCAL_DIR   = os.path.join(DIR_MAIN, 'data/dataset/VOCdevkit/')
        PASCAL_TRAIN = os.path.join(DIR_MAIN, 'data/dataset/VOCdevkit/voc_train.txt')
        PASCAL_VALID = os.path.join(DIR_MAIN, 'data/dataset/VOCdevkit/2007_test.txt')
        TRAIN_LOGDIR = os.path.join(DIR_MAIN, 'train_data')
        VAL_LOGDIR   = os.path.join(DIR_MAIN, 'eval_data')
        VAL_OUTPUTDIR_PKL = os.path.join(DIR_MAIN, 'eval_results')
        MODEL_CFG    = os.path.join(DIR_MAIN, 'data/cfg/github_pjreddie/yolov2-voc.cfg')
        
    if (1):
        
        #MODEL_WEIGHT = os.path.join(DIR_MAIN, 'data/weights/yolov2-voc.weights')
        #VAL_PREFIX   = 'pretrained_'
        
        MODEL_WEIGHT = os.path.join(DIR_MAIN, 'data/weights/retrain-pretrain/filter-pruned-0.0-retrained_000015.weights')
        VAL_PREFIX   = 'retrain_pretrain_'
        
        print (' - 0. MODEL_WEIGHT :  ', MODEL_WEIGHT)
        print (' - 0. VAL_PREFIX   : ', VAL_PREFIX)
        
    if (1):
        BATCH_SIZE    = 32;
        print (' - 0. BATCH_SIZE    : ', BATCH_SIZE)
        
    if (1):
        DEBUG_EPOCHS = -1 #[-1, 50]
        MAX_EPOCHS   = 100
        
        print (' - 0. DEBUG_EPOCHS : ', DEBUG_EPOCHS)
        print (' - 0. MAX_EPOCHS   : ', MAX_EPOCHS)

    if (1):
        LEARNING_RATE = 0.0001
        print (' - 0. LEARNING_RATE : ', LEARNING_RATE)
        LEARNING_RATES = [LEARNING_RATE for _ in range(MAX_EPOCHS)]
    else:
        import numpy as np
        a = np.linspace(0.0005,0.0001, 10)
        b = np.linspace(0.0001, 0.00001, 10)
        c = np.linspace(0.00001, 0.0001, 10)
        d = np.linspace(0.0001, 0.0005,10) 
        LEARNING_RATES = np.hstack((a,b,c,d,a,b,c,d,a,b))

        import matplotlib.pyplot as plt
        plt.plot(LEARNING_RATES)
        plt.title('Cyclical Learning Rates')
        plt.xlabel('Epochs')
        plt.ylabel('Learning Rate')
        plt.show()
    
    assert len(LEARNING_RATES) == MAX_EPOCHS
    
    if (1):
        pruning_perc   = 80. #[0, 10., 30., 50., 70., 90.]
        pruning_method = "weight" # ["filter", "weight"]
    
    if (1):
        try:
            LOGGER = TensorBoardColab()
            print (' - 0. Logger       : ', LOGGER)
        except:
            LOGGER = ''
            print (' - 0. Logger       : ', LOGGER)
        print ('')
    

    if (1):
        trainObj = YOLOv2Train()
        trainObj.train(PASCAL_DIR, PASCAL_TRAIN, PASCAL_VALID, TRAIN_LOGDIR, VAL_LOGDIR, VAL_OUTPUTDIR_PKL, VAL_PREFIX
                       , MODEL_CFG, MODEL_WEIGHT
                       , BATCH_SIZE, LEARNING_RATES, MAX_EPOCHS
                       , LOGGER, DEBUG_EPOCHS=DEBUG_EPOCHS, verbose=0
                       , pruning_perc=pruning_perc, pruning_method=pruning_method
                      )
else:
    print (' - GPU Issues!!')

In [0]:
# Zips all grad_flow images together
GRAD_FLOW_DIR = os.path.join(TRAIN_LOGDIR, "grad_flow")
GRAD_FLOW_ZIP_DEST = os.path.join(TRAIN_LOGDIR, "grad_flow.zip")
! zip -r {GRAD_FLOW_ZIP_DEST} {GRAD_FLOW_DIR}